In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
x = np.random.random(15)
y = np.random.random(15)

In [3]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [4]:
data = [[i, j, FrankeFunction(i, j)] for i in x for j in y]

In [5]:
data = np.array(data)

In [6]:
data[:,2] += np.random.random(size=data[:,2].shape)

In [7]:
X, y = np.hsplit(data, [2])

In [8]:
np.polynomial.polynomial.polygrid2d([1,2], [2,3], np.array([[0,1],[0,1]]))

array([[4., 6.],
       [6., 9.]])

In [9]:
def PolyTransformer(X, degree=5):
    transformedX = list()
    for x,y in X:
        transformedSample = list()
        for p in range(1, degree+1):
            for n in range(p+1):
                transformedSample.append(x**(p-n) * y**n)
        transformedX.append(transformedSample)
    return np.array(transformedX)

In [10]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=.8)

In [11]:
train_X = PolyTransformer(train_X)

In [12]:
scaler = StandardScaler()

In [13]:
train_X = scaler.fit_transform(train_X)

In [14]:
beta = train_X.T @ train_X # beta will evolve according to the algorithm

In [15]:
U,S,V = np.linalg.svd(beta, hermitian=True)

# SVD is successful

In [16]:
np.allclose(U @ np.diag(S) @ V, beta)

True

In [17]:
invBeta = V.T @ np.diag(1/S) @ U.T

# Inversion is successful

In [18]:
np.allclose(invBeta, np.linalg.inv(beta))

True

In [19]:
beta = invBeta

In [20]:
beta = beta @ train_X.T

In [21]:
beta = beta @ train_y

In [22]:
def mse(y_true, y_pred):
    return ((y_true - y_pred)**2).mean()

In [23]:
def r_2(y_true, y_pred):
    return 1 - ((y_true - y_pred)**2).sum() / ((y_true - y_true.mean())**2).sum()

In [31]:
mse(train_X @ beta, train_y)

0.919682615384677

In [32]:
r_2(train_X @ beta, train_y)

-8.397667699308473

In [33]:
mse(test_y, scaler.transform(PolyTransformer(test_X)) @ beta)

0.797413212056113

In [34]:
r_2(test_y, scaler.transform(PolyTransformer(test_X)) @ beta)

-4.554952017514757